# Einführung Pandas

**Inhalt:** Ein Streifzug durch die Möglichkeiten von Pandas

**Nötige skills**: Keine

**Lerniziele**:
- Übersicht erhalten: Nur schauen, nicht selber programmieren
- Anhand eines konkreten Beispiels einen Workflow beobachten
- Erkennen, welche Arten von Operationen und Outputs Pandas hat

### Was ist Pandas?
Kurz gesagt: Pandas ist Excel für Python.

Das Modul ermöglicht ähnliche Operationen wie ein Tabellenkalkulationsprogramm, aber
- in Logbuchform statt als Tabellenblatt
- mit reproduzierbarem Code
- viel mehr Funktionalität
- kombinierbar mit anderen Modulen

Ohne Pandas (fast) kein Datenjournalismus mit Python.

### Installation
Command line: `pip install pandas`

...

...

... und wenn das alles geklappt hat:

In [ ]:
import pandas as pd

In [ ]:
%matplotlib inline

In [ ]:
print ("Hooray!")

# Das Beispiel

Eine Datenbank zu allen militärischen Auseinandersetzungen zwischen 1816 und 2010.

Quelle: http://cow.dss.ucdavis.edu/data-sets/MIDs

## Datenfile öffnen

Der notwendige Start vor jeder Datenanalyse.

In [ ]:
df = pd.read_csv('dataprojects/military-interventions/MID-level/MIDB_4.2.csv')

## Basic exploration

Die Grösse und Struktur der Datenbank: Sollte beim öffnen jedes Files zuerst einmal grob angeschaut werden.

### Wie viele Einträge hat die Datenbank?

In [ ]:
df.shape

### Was für Datenfelder kommen darin vor?

In [ ]:
df.dtypes

Für Details zu den Codes, check:

`dataprojects/Military Interventions/MID-level/MID_v4.0_Codebook.pdf`

### Wie muss man sich die Einträge in dieser Tabelle ungefähr vorstellen?

In [ ]:
df.head(3)

## Spontane Fragen

Die ersten Dinge, die einem zum Thema militärische Auseinandersetzungen in den Sinn kommen.

Mit Pandas lassen sie sich relativ rasch beantworten, falls die Daten bereits gut aufbereitet, vollständig und ohne viele Codierfehler da sind.

### Welche Länder haben am meisten militärische Auseinandersetzungen teilgenommen?

In [ ]:
df.stabb.value_counts().head(10)

In [ ]:
df.stabb.value_counts().head(20).plot(kind='bar')

### Zu welcher Zeit gab es am meisten Konflikte?

In [ ]:
df.styear.value_counts().sort_index().plot()

### Um was für Konflikte handelt es sich?

In [ ]:
hostlev_codes = pd.Series({
    0: "Unknown",
    1: "No militarized action",
    2: "Threat to use force",
    3: "Display of force",
    4: "Use of force",
    5: "War"
})

In [ ]:
df_hostlev_codes = hostlev_codes.to_frame().rename(columns={0: "intensity"})

In [ ]:
df.hostlev.value_counts().to_frame().join(df_hostlev_codes).sort_index()

In [ ]:
df.hostlev.value_counts().to_frame().join(df_hostlev_codes).sort_index().plot(kind='barh', y="hostlev", x="intensity")

Wichtig: mehrere Wege führen zum Ziel. Bsp oben, man könnte den Code auch anders schreiben...

## Wir wollen etwas mehr wissen

### Zeitdauer von verschiedenen Konflikttypen

In [ ]:
#create one date field out of year/month/day for start and end date
df['stdate'] = df.styear.astype(str) + "/" + df.stmon.astype(str) + "/" + df.stday.astype(str)
df['enddate'] = df.endyear.astype(str) + "/" + df.endmon.astype(str) + "/" + df.endday.astype(str)

In [ ]:
#oops, some days have "-9" in them (=unknown dates). replace them with 15 (best guess)
df['stdate'] = df.stdate.str.replace("-9","15")
df['enddate'] = df.enddate.str.replace("-9","15")

In [ ]:
#convert to datetime
df['stdate'] = pd.to_datetime(df.stdate, format="%Y/%m/%d")
df['enddate'] = pd.to_datetime(df.enddate, format="%Y/%m/%d")

In [ ]:
#new field for the time difference
df['timediff'] = df.enddate - df.stdate

In [ ]:
#get back the days and months
df['timediff_days'] = df.timediff.dt.days

In [ ]:
plt = df.timediff_days.hist(bins=100, figsize=(10,5))
plt.set_title("Anzahl der Konflikte nach Zeitdauer, in Tagen")

In [ ]:
df.timediff.describe()

### Welche Länder haben Konflikte angefangen?

In [ ]:
#field 'orig' has 1 if country 'originated' the conlfict (whatever this means)
df.groupby('stabb')['orig'].mean().sort_values(ascending=False)

In [ ]:
#look only at the 10 countries with most conflicts
top10 = df.stabb.value_counts().head(10).keys()

In [ ]:
df[df.stabb.isin(top10)].groupby('stabb')['orig'].mean().sort_values(ascending=False)

**Wer zur Hölle ist IRN?**

-> Die Auswertung ist nur so gut wie die Info, die in der Datenbank drin ist
-> Pandas selbst beantwortet in den seltensten Fällen alles

In [ ]:
df_irn = df[df.stabb == 'IRN']
df_irn.head(3)

IRN has country code 630
- could be Iran
- or Puerto Rico ??

In [ ]:
df_irn.styear.value_counts().sort_index().plot()

Looks like it is Iran after all...

### Was für Konflikte hat Iran geführt?

In [ ]:
#We still have our hostlev_codes...
df_hostlev_codes

In [ ]:
df_irn.hostlev.value_counts().to_frame().join(df_hostlev_codes).sort_index()

In [ ]:
df_irn.hostlev.value_counts().to_frame().join(df_hostlev_codes).sort_index().plot(kind='barh', y="hostlev", x="intensity")

### Wie viele Tote gab es dabei?

In [ ]:
fatality_codes = pd.Series({
    0: "None",
    1: "1-25 deaths",
    2: "26-100 deaths",
    3: "101-250 deaths",
    4: "251-500 deaths",
    5: "501-999 deaths",
    6: "More than 999 deaths",
    -9: "Unknown"
})

In [ ]:
df_fatality_codes = fatality_codes.to_frame().rename(columns={0: "fatalities"})
df_fatality_codes

In [ ]:
df_irn_fatality = df_irn.fatality.value_counts().to_frame().rename(columns={'fatality': 'number'})
df_irn_fatality

In [ ]:
df_irn_fatality.join(df_fatality_codes, how="outer").sort_index().fillna(0)

In [ ]:
df_irn_fatality.join(df_fatality_codes, how="outer").sort_index().fillna(0).plot(kind='barh', y="number", x="fatalities")

Iran hat zwei Kriege mit über 999 Toten geführt, bei den meisten Konflikten gab es aber keine Toten.

## Daten exportieren und anderweitig verwenden

Sagen wir mal, wir schreiben gerade einen Artikel über die Geschichte der Kriege.
- Nehmen wir an, wir möchten die Grafik online mit 'infogram' weiterverbreiten
- Wir möchten für eine interaktive Web-Grafk aufzeigen, welche Länder in wie viele Kriege verwickelt waren
- Uns interessieren die 10 Länder, die am häufigsten in den Hostility Kategorien 4 und 5 drin waren
- Wir wollen das 19. und das 20. Jahrhundert separat anschauen

### Daten vorbereiten

In [ ]:
# Select only level 4 and 5 types
df_45 = df[df.hostlev.isin([4, 5])]

In [ ]:
# Which are the top 10 countries?
top10 = df_45.stabb.value_counts().head(10).keys()
top10

In [ ]:
# Select only the top 10 countries
df_45_top10 = df_45[df.stabb.isin(top10)].copy()

In [ ]:
# Create a boolean column if a conflict was in the 19th century
df_45_top10['19th'] = df_45_top10.styear < 1901
df_45_top10.head(3)

In [ ]:
# Create another boolean column if the conflict wasnt in the 19th
df_45_top10['20th'] = ~df_45_top10['19th']

In [ ]:
# Transform the boolean columns to integers
df_45_top10['c19th'] = df_45_top10['19th'].astype(int)
df_45_top10['c20th'] = df_45_top10['20th'].astype(int)
df_45_top10.head(3)

In [ ]:
# Group by country and sum up the integer columns for both centuries, sort
df_chart = df_45_top10.groupby('stabb')[['19th', '20th']].sum().astype(int).reset_index().sort_values('19th', ascending=False)
df_chart

### Daten exportieren

In [ ]:
df_chart.to_csv('war-chart.csv', index=False)

### Das steht in unserem csv-File:

In [ ]:
with open('war-chart.csv', "r") as f:
    file_content = f.read()

In [ ]:
print(file_content)

### Online-Chart basteln

Go to http://infogram.com

Paste the csv data...

Results: https://infogram.com/top-10-war-countries-1h0n25ger7jz6pe?live